In [3]:
from langchain_core.messages import AnyMessage
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt import create_react_agent
from langgraph.prebuilt.chat_agent_executor import AgentState

In [1]:
def get_weather(city: str) -> str:
    """ Get weather for a given city."""
    return f"It's always sunny in {city}"

In [7]:
def prompt(state: AgentState, config: RunnableConfig) -> list[AnyMessage]:
    user_name = config['configurable'].get("user_name")
    system_prompt = f"YOu are a helpful assistant. User name is {user_name}"
    return [{"role": "system", "content": system_prompt}] + state["messages"]

In [8]:
agent = create_react_agent(
    model='gpt-4o',
    tools=[get_weather],
    prompt=prompt
)

In [9]:
agent.invoke(
    {"messages": [{"role": "user", "content": "hi"}]},
    config={"configurable": {"user_name": "John Smith"}}
)

{'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='22eae9a9-4f47-4a6e-a0b3-1803b2c8fc3c'),
  AIMessage(content='Hello, John! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 58, 'total_tokens': 70, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_90122d973c', 'id': 'chatcmpl-BYtVLdarNjaaDcy12DDMdQy4INr59', 'finish_reason': 'stop', 'logprobs': None}, id='run-d4396ef7-4573-453c-8c0d-addf47841c9b-0', usage_metadata={'input_tokens': 58, 'output_tokens': 12, 'total_tokens': 70, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

In [10]:
from typing import Annotated
from langgraph.prebuilt import InjectedState

In [11]:
class CustomState(AgentState):
    user_id : str

In [13]:
def get_user_info(state: Annotated[CustomState, InjectedState]) -> str:
    """ Look up user info """
    user_id = state["user_id"]
    return "User is John Smith" if user_id == "user_123" else "Unknown User"

In [14]:
aget_state = create_react_agent(
    model='gpt-4o',
    tools=[get_user_info],
    state_schema=CustomState
)